<a href="https://colab.research.google.com/github/henrygas/tensorflow_2_learn/blob/master/tensor_homework_senior_units_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

1.15.0-rc3
1.16.5


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

mnist = input_data.read_data_sets("./")

print(mnist.train.images.shape)
print(mnist.train.labels.shape)

print(mnist.validation.images.shape)
print(mnist.validation.labels.shape)

print(mnist.test.images.shape)
print(mnist.test.labels.shape)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
(55000, 784)
(55000,)
(5000, 784)
(5000,)
(10000, 784)
(10000,)


In [0]:
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("int64", [None])
learning_rate = tf.placeholder("float")

L1_units_count = 500
W_1 = tf.Variable(tf.truncated_normal([784, L1_units_count], stddev=0.1))
b_1 = tf.Variable(tf.truncated_normal([L1_units_count], stddev=0.1))
logits_1 = tf.matmul(x, W_1) + b_1
output_1 = tf.nn.relu(logits_1)

L2_units_count = 10
W_2 = tf.Variable(tf.truncated_normal([L1_units_count, L2_units_count], stddev=0.1))
b_2 = tf.Variable(tf.truncated_normal([L2_units_count], stddev=0.1))
logits_2 = tf.matmul(output_1, W_2) + b_2

logits = logits_2

cross_entropy_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy_loss)

pred = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(pred, 1), y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

batch_size = 32
learning_rate_value = 0.5
max_train_step = 1000
saver = tf.train.Saver()

acc_train_max = 0
acc_validation_max = 0
acc_test_max = 0

test_times = 30
while (test_times > 0):
  test_times -= 1
  with tf.Session() as ses:
    ses.run(tf.global_variables_initializer())
    for i in range(max_train_step):
      xs, ys = mnist.train.next_batch(batch_size)
      _, loss, W_1_value, b_1_value, W_2_value, b_2_value = ses.run([optimizer, cross_entropy_loss, W_1, b_1, W_2, b_2], feed_dict={
          x: xs,
          y: ys,
          learning_rate: learning_rate_value
      })

    acc_train = ses.run(accuracy, feed_dict={
        x: mnist.train.images,
        y: mnist.train.labels
    })
    if acc_train > acc_train_max:
      acc_train_max = acc_train
      saver.save(ses, "./train_model.ckpt", global_step=max_train_step)

    acc_validation = ses.run(accuracy, feed_dict={
        x: mnist.validation.images,
        y: mnist.validation.labels
    })
    if acc_validation > acc_validation_max:
      acc_validation_max = acc_validation
      saver.save(ses, "./validation_model.ckpt", global_step=max_train_step)

    acc_test = ses.run(accuracy, feed_dict={
        x: mnist.test.images,
        y: mnist.test.labels
    })
    if acc_test > acc_test_max:
      acc_test_max = acc_test
      saver.save(ses, "./test_model.ckpt", global_step=max_train_step)

In [4]:
with tf.Session() as ses:
  train_model = saver.restore(ses, "./train_model.ckpt" + "-" + str(max_train_step))
  acc_train = ses.run(accuracy, feed_dict={
      x: mnist.train.images,
      y: mnist.train.labels
  })
  acc_validation = ses.run(accuracy, feed_dict={
      x: mnist.validation.images,
      y: mnist.validation.labels
  })
  acc_test = ses.run(accuracy, feed_dict={
      x: mnist.test.images,
      y: mnist.test.labels
  })
  print("using train_model, acc_train:{}, acc_validation:{}, acc_test:{}".format(acc_train, acc_validation, acc_test))


INFO:tensorflow:Restoring parameters from ./train_model.ckpt-1000
using train_model, acc_train:0.9673636555671692, acc_validation:0.9639999866485596, acc_test:0.9614999890327454


In [5]:
with tf.Session() as ses:
  validation_model = saver.restore(ses, "./validation_model.ckpt" + "-" + str(max_train_step))
  acc_train = ses.run(accuracy, feed_dict={
      x: mnist.train.images,
      y: mnist.train.labels
  })
  acc_validation = ses.run(accuracy, feed_dict={
      x: mnist.validation.images,
      y: mnist.validation.labels
  })
  acc_test = ses.run(accuracy, feed_dict={
      x: mnist.test.images,
      y: mnist.test.labels
  })
  print("using validation_model, acc_train:{}, acc_validation:{}, acc_test:{}".format(acc_train, acc_validation, acc_test))

INFO:tensorflow:Restoring parameters from ./validation_model.ckpt-1000
using validation_model, acc_train:0.9642181992530823, acc_validation:0.9660000205039978, acc_test:0.958299994468689


In [6]:
with tf.Session() as ses:
  test_model = saver.restore(ses, "./test_model.ckpt" + "-" + str(max_train_step))
  acc_train = ses.run(accuracy, feed_dict={
      x: mnist.train.images,
      y: mnist.train.labels
  })
  acc_validation = ses.run(accuracy, feed_dict={
      x: mnist.validation.images,
      y: mnist.validation.labels
  })
  acc_test = ses.run(accuracy, feed_dict={
      x: mnist.test.images,
      y: mnist.test.labels
  })
  print("using test_model, acc_train:{}, acc_validation:{}, acc_test:{}".format(acc_train, acc_validation, acc_test))

INFO:tensorflow:Restoring parameters from ./test_model.ckpt-1000
using test_model, acc_train:0.9654545187950134, acc_validation:0.9599999785423279, acc_test:0.9624000191688538
